<div class="alert alert-block alert-info">
<h1 style="text-align:center;font-family:avenir;">GLM t-test</h1>
Lou Planchamp & Victor Férat
</div>

In [ ]:
%matplotlib inline

In [ ]:
from nilearn import plotting
from nilearn.plotting import plot_stat_map
import pandas as pd
from nilearn.glm.first_level import FirstLevelModel
from numpy import array
import os
from nilearn.plotting import plot_design_matrix
imporbt numpy as np
import matplotlib.pyplot as plt
from nilearn.glm.second_level import SecondLevelModel

This is just a function used later to save the lists of first t maps in png

In [ ]:
def render_mpl_table(data, col_width=3.0, row_height=0.625, font_size=14,
                     header_color='#40466e', row_colors=['#f1f1f2', 'w'], edge_color='w',
                     bbox=[0, 0, 1, 1], header_columns=0,
                     ax=None, **kwargs):
    if ax is None:
        size = (np.array(data.shape[::-1]) + np.array([0, 1])) * np.array([col_width, row_height])
        fig, ax = plt.subplots(figsize=size)
        ax.axis('off')
    mpl_table = ax.table(cellText=data.values, bbox=bbox, colLabels=data.columns, **kwargs)
    mpl_table.auto_set_font_size(False)
    mpl_table.set_fontsize(font_size)

    for k, cell in mpl_table._cells.items():
        cell.set_edgecolor(edge_color)
        if k[0] == 0 or k[1] < header_columns:
            cell.set_text_props(weight='bold', color='w')
            cell.set_facecolor(header_color)
        else:
            cell.set_facecolor(row_colors[k[0]%len(row_colors) ])
    return ax.get_figure(), ax

Run this code to compute a t-test between pre and post session with all subjects

In [ ]:
# ALL sub
anat = '/home/carole/Bureau/Lou_dualreg/tpl-MNIPediatricAsym_cohort-3_res-1_T1w.nii'

if not os.path.isdir('t_test_GLM'): 
    os.mkdir('t_test_GLM')
if not os.path.isdir('t_test_GLM/fig'): 
    os.mkdir('t_test_GLM/fig')
if not os.path.isdir('t_test_GLM/zmaps'): 
    os.mkdir('t_test_GLM/zmaps')
df_folder = f't_test_GLM/fig/list_t_maps'
if not os.path.isdir(df_folder): 
    os.mkdir(df_folder)

for task in ['Calcul', 'CPT', 'Memory']:
    ds = list()
    for session in ['02', '11']:
        print(task)
        print(session)
        
        first_t_maps = dict()
        folder = f'first_level_GLM/tmaps/{task}'
        tmaps = []
        for file in os.listdir(folder):
            if file.endswith(f'{session}_{task}_tmap.nii'):
                d = {
                    "session": session,
                    "subject": file.split("_")[0],
                    "img": os.path.join(folder,file)
                }
                ds.append(d)
                
    df = pd.DataFrame(ds)
    fig,ax = render_mpl_table(df, header_columns=0, col_width=10.0)
    fig.savefig(f't_test_GLM/fig/list_t_maps/list_t_maps_{task}_all-sub.png')

#===================================================================
    
    columns = ["session"] + df["subject"].unique().tolist()

    M = np.zeros((len(df),len(columns)))

    for r,row in df.iterrows():
        # session
        if row["session"] == '02':
            M[r,0] = -1
        else:
            M[r,0] = 1
        # subjects
        for c,column in enumerate(columns):
            if row["subject"] == column:
                M[r,c] = 1

    files = df["img"].tolist()  
    design_matrix = pd.DataFrame(M, columns=columns)
    
    dm_folder = f't_test_GLM/fig/design_matrices'
    if not os.path.isdir(dm_folder): 
        os.mkdir(dm_folder)
    
    plot_design_matrix(design_matrix,output_file=f't_test_GLM/fig/design_matrices/design_matrix_t-test_{task}_all-sub.png')

    second_level_input = files    
    second_level_model = SecondLevelModel(smoothing_fwhm=8.0)
    second_level_model = second_level_model.fit(
        second_level_input,
        design_matrix=design_matrix)

    sec_z_map = second_level_model.compute_contrast("session", output_type='z_score')
    
    zpath = f't_test_GLM/zmaps/z-map_t-test_all-sub_{task}.nii'
    sec_z_map.to_filename(zpath)

    sec_z_maps_folder = f't_test_GLM/fig/z-maps_t-test_all-sub'
    if not os.path.isdir(sec_z_maps_folder): 
        os.mkdir(sec_z_maps_folder)
    zfig_name = f't_test_GLM/fig/z-maps_t-test_all-sub/fig_z-map_t-test_all-sub_{task}.png'

    plotting.plot_stat_map(sec_z_map, display_mode='mosaic',
                           threshold=2, bg_img=anat, 
                           output_file=zfig_name,
                           annotate=False, black_bg=True, cut_coords=8,
                           title=f'z-maps t-test {task} all-sub')


Run this code to compute a t-test between pre and post session without subjects 5 and 7

In [ ]:
# WO sub 5 and 7
anat = '/home/carole/Bureau/Lou_dualreg/tpl-MNIPediatricAsym_cohort-3_res-1_T1w.nii'

if not os.path.isdir('t_test_GLM'): 
    os.mkdir('t_test_GLM')
if not os.path.isdir('t_test_GLM/fig'): 
    os.mkdir('t_test_GLM/fig')
if not os.path.isdir('t_test_GLM/zmaps'): 
    os.mkdir('t_test_GLM/zmaps')
df_folder = f't_test_GLM/fig/list_t_maps'
if not os.path.isdir(df_folder): 
    os.mkdir(df_folder)

for task in ['Calcul', 'CPT', 'Memory']:
    ds = list()
    for session in ['02', '11']:
        print(task)
        print(session)
        
        first_t_maps = dict()
        folder = f'first_level_GLM/tmaps/{task}'
        tmaps = []
        for file in os.listdir(folder):
            if file.endswith(f'{session}_{task}_tmap.nii'):
                if not file.startswith('sub-05') and not file.startswith('sub-07'):
                    d = {
                        "session": session,
                        "subject": file.split("_")[0],
                        "img": os.path.join(folder,file)
                    }
                    ds.append(d)
                
    df = pd.DataFrame(ds)
    fig,ax = render_mpl_table(df, header_columns=0, col_width=10.0)
    fig.savefig(f't_test_GLM/fig/list_t_maps/list_t_maps_{task}_wo-sub.png')

#===================================================================
    
    columns = ["session"] + df["subject"].unique().tolist()

    M = np.zeros((len(df),len(columns)))

    for r,row in df.iterrows():
        # session
        if row["session"] == '02':
            M[r,0] = -1
        else:
            M[r,0] = 1
        # subjects
        for c,column in enumerate(columns):
            if row["subject"] == column:
                M[r,c] = 1

    files = df["img"].tolist()  
    design_matrix = pd.DataFrame(M, columns=columns)
    
    dm_folder = f't_test_GLM/fig/design_matrices'
    if not os.path.isdir(dm_folder): 
        os.mkdir(dm_folder)
    
    plot_design_matrix(design_matrix,output_file=f't_test_GLM/fig/design_matrices/design_matrix_t-test_{task}_wo-sub.png')

    second_level_input = files    
    second_level_model = SecondLevelModel(smoothing_fwhm=8.0)
    second_level_model = second_level_model.fit(
        second_level_input,
        design_matrix=design_matrix)

    sec_z_map = second_level_model.compute_contrast("session", output_type='z_score')
    
    zpath = f't_test_GLM/zmaps/z-map_t-test_wo-sub_{task}.nii'
    sec_z_map.to_filename(zpath)

    sec_z_maps_folder = f't_test_GLM/fig/z-maps_t-test_wo-sub'
    if not os.path.isdir(sec_z_maps_folder): 
        os.mkdir(sec_z_maps_folder)
    zfig_name = f't_test_GLM/fig/z-maps_t-test_wo-sub/fig_z-map_t-test_wo-sub_{task}.png'

    plotting.plot_stat_map(sec_z_map, display_mode='mosaic',
                           threshold=2, bg_img=anat, 
                           output_file=zfig_name,
                           annotate=False, black_bg=True, cut_coords=8,
                           title=f'z-maps t-test {task} wo-sub')
